In [2]:
#ELS-1underlying
import numpy as np
from datetime import date

In [3]:
# 시뮬레이션 10000, 이자율 1.65, 변동성 17.78%
n = 10000 ; r = 0.0165 ; vol = 0.1778

In [25]:
# 체크 날짜 및 행사가격 설정
n0 = date.toordinal(date(2018,3,23))
n1 = date.toordinal(date(2018,9,19))
n2 = date.toordinal(date(2019,3,20))
n3 = date.toordinal(date(2019,9,23))
n4 = date.toordinal(date(2020,3,23))
n5 = date.toordinal(date(2020,9,23))
n6 = date.toordinal(date(2021,3,23))
check_day = np.array([n1-n0,n2-n0,n3-n0,
                     n4-n0,n5-n0,n6-n0])
oneyear = 365 ; total_date = n6-n0 
dt = 1/oneyear
S = np.zeros([total_date+1, 1])
S[0] = 100.0
strike_price = np.array([0.95, 0.95, 0.95, 0.90, 0.90, 0.85])*S[0]

In [19]:

repay_n = len(strike_price)
coupon_rate = np.array([0.022, 0.044, 0.066, 0.088,
                       0.11, 0.132])
payment = np.zeros([repay_n, 1])

In [20]:
#액면가, payoff등
facevalue = 10**4
total_payoff = np.zeros([repay_n, 1])
payoff = np.zeros([repay_n, 1])
discount_payoff = np.zeros([repay_n, 1])
knock_in_barrier = S[0]*0.65 ; dummy = 0.132

In [21]:
#조기상환 했을 때 벡터 생성
for j in range(repay_n):
    payment[j] = facevalue*(1+coupon_rate[j])

In [27]:
#
for i in range(n):
    z = np.random.normal(0, 1, size = [total_date, 1])
    for j in range(total_date):
        S[j+1] = S[j]*np.exp((r-0.5*vol**2)*dt+vol*np.sqrt(dt)*z[j])
    S_checkday = S[check_day]
    payoff = np.zeros([repay_n, 1]); repay_event = 0
    for j in range(repay_n):
        if S_checkday[j] >= strike_price[j]:
            payoff[j] = payment[j]
            repay_event = 1
            break
    if repay_event == 0:
        if min(S) > knock_in_barrier:
            payoff[-1] = facevalue*(1+dummy)
        else:
            payoff[-1] = facevalue*(S[-1]/S[0])
    total_payoff = total_payoff + payoff
mean_payoff = total_payoff/n
for j in range(repay_n):
    discount_payoff[j] = mean_payoff[j]*np.exp(-r*check_day[j]/oneyear)
price = np.sum(discount_payoff)
print(price)

9877.687695673761
